# 4.8_Grouping Data and Aggregating Variables

## Contents
### Importing Libraries
### Importing Data
### Path to Storage
### Grouping Data with Pandas
### Aggregating Data with agg()
### Aggregating Data with transform()
### Deriving Columns with loc()
### Export Data

### Importing Libraries

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os

### Path to Storage

In [ ]:
# Path to storage location
path = r'C:\Users\ihs10539\Achievement 4 Project'

### Importing Data

In [ ]:
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02_Data', '02.02_Prepared Data', '4.7_Task_df_ords_prods_merged_derived.pkl'))

In [ ]:
# Subset
df_ords_prods_merge_subset = df_ords_prods_merge[:1000000]

In [ ]:
# Verify Subset
df_ords_prods_merge_subset.shape

In [ ]:
df_ords_prods_merge_subset.head(10)

## Grouping Data with Pandas

In [ ]:
# Grouping data in pandas follows a set of logical steps
# First, split the data into groups based on given criteria
# Next, apply a function to these groups
# Lastly, combine the results to see what was created

In [ ]:
# The 'groupby()' function will group a given dataframe by a given column
df_ords_prods_merge_subset.groupby('product_name')

In [ ]:
# The function created a pandas object
# However, the output is not visible yet
# Something else needs to be done, like aggregating the data or applying a function, before seeing the results
# groupby() should always be used as part of a series of steps

In [ ]:
# So far we've split the data into groups using the 'product_name' column
# The next step will involve some aggregation

## Aggregating Data with agg()

In [ ]:
# Aggregating functions create summaries of selected columns and store these summary values in new columns
# These summaries usually take the form of a descriptive statistic

In [ ]:
# The 'agg()' function is used for aggregation
# We can use this function to produce a single descriptive statistic for the 'order_number' column
# If you were to calculate the mean of the 'order_number' column grouped by the 'department_id' column, you can compare user orders in each department
# 'order_number' refers to the number of orders placed by a given user
# The process is as follows:
# 1. Split the data into groups based on 'department_id'
# 2. Apply the agg() function to each group to obtain the mean values for the 'order_number' column
df_ords_prods_merge_subset.groupby('department_id').agg({'order_number': ['mean']})

In [ ]:
# The groupby() function is assigned to the df dataframe, creating a pandas object for the 'department_id'
# The agg() function is applied to this object, returning the mean of the given column 'order_id'
# The result is the average number of orders per user for each department id
# Despite the results, it's important to conduct statistical tests to check for a significant difference between the means of the different grouos
# One such test is the 't-test'.

In [ ]:
# There are some aggregations that can be conducted without the use of the agg() function
df_ords_prods_merge_subset.groupby('department_id')['order_number'].mean()

In [ ]:
# The difference in syntex between the two methods:
# 1. When using agg(), put the column you want to aggregate inside the parenthesis of the agg()
# 2. When using mean(), simply index the column with square brackets, then follow it with the function you want to use after the dot

In [ ]:
# Dot notation vs. square brackets
# df.groupby('department_id').order_number.mean()
# Using dot notation results in the same output , however, there are reasons you shouldn't
# 1. Square brackets stand out and are more readable 
# 2. Square brackets have no other role in Python beyond indexing, so using dot notation can make it slower since it has to work harder to understand

In [ ]:
# You can perform multiple aggregations at once
df_ords_prods_merge_subset.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

## Aggregating Data with transform()

In [ ]:
# We'll be creating 'loyalty' flags if our dataframe
# 'Loyalty' customers are those who come back time and time again to use the service or buy products
# We've been tastked to locate these customers so that the business strategy team can employ some kind of bonus point program for them
# This can be achieved with the 'transform()' function, which will create a new column containing the maximum frequency of the 'order_number' column
# Then, using the loc() function, a second column will be created containing a flag designating whether a customer is 'loyal' or not

In [ ]:
# To create the flag, we'll need some criteria:
# 1. If the maximum orders the user has made is over 40, then the customer will be labeled a 'Loyal customer'
# 2. If the maximum orders the user has made is over 10 but less than or equal to 40, then the customer will be labeled a 'Regular customer'
# 3. If the maximum orders the user has made is less than or equal to 10, then the customer will be labeled a 'New customer'

In [ ]:
# Here's the steps to accomplish this
# 1. Split the data in groups based on the 'user_id' column
# 2. Apply the trasnform() function on the 'order_number' column to generate the maximum orders for each user
# 3. Create a new column, 'max_order,' into which you'll place the results of your aggregation
# Once this process is complete we can use the 'max_order' column to create a new column that assigns a loyalty flag to each customer using loc()
df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform('max')

In [ ]:
df_ords_prods_merge.head(15)

In [ ]:
# Now we have a new 'max_order' column in our dataframe
# Each value in this column corresponds to the maximum number of orders made by each user id

In [ ]:
# You can check the output by printing the head of the dataframe with an argument of, say 100
df_ords_prods_merge.head(100)

In [ ]:
# However, the current settings won't allow you to see all 100
# This can be changed via the following code:
pd.options.display.max_rows = None

In [ ]:
# Now, all 100 rows we called for can be viewed

## Deriving Columns with loc()

In [ ]:
# Now that the new column is created, we must flag customers who meet the criteria as 'loyal'
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [ ]:
df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] >10), 'loyalty_flag'] = 'Regular customer'

In [ ]:
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [ ]:
# Check value counts for new column
df_ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

In [ ]:
# Most customers fall into the 'Regular customer' category 

In [ ]:
# As always, check that everything was interpreted correctly and that the right flags were assigned 
df_ords_prods_merge[['user_id', 'loyalty_flag', 'order_number']].head(60)

## Export Data

In [ ]:
# Export dataframe with newly derived columns

In [ ]:
df_ords_prods_merge.to_pickle(os.path.join(path, '02_Data', '02.02_Prepared Data', '4.8_Task_df_ords_prods_merged_derived_GDAV.pkl'))

In [ ]:
df_ords_prods_merge.shape